# Tune LR, SVC, LightGBM

In [ ]:
import sys
import json

sys.path.append("../")
from src.config import BASE_PATH
from src.data_utils import get_data
from src.tune import (
    lr_model_builder,
    lightgbm_model_builder,
    svc_model_builder,
    tune_model_mult_outcomes,
    get_prelim_results,
)

from joblib import Parallel, delayed

Set globals

In [ ]:
OUTCOME_DICT = {
    "surg": get_data("outcome_surg"),
    "bleed": get_data("outcome_bleed"),
    "asp": get_data("outcome_asp"),
    "mort": get_data("outcome_mort"),
    "reop": get_data("outcome_reop"),
}

NUM_TRIALS = 300
LOG_PATH = BASE_PATH / "logs"
RESULT_PATH = BASE_PATH / "models" / "tune_results"

Call Tuner

In [ ]:
jobs = [
    delayed(tune_model_mult_outcomes)(
        model_builder=lr_model_builder,
        model_abrv="lr",
        outcome_dict=OUTCOME_DICT,
        scoring="roc_auc",
        log_file_path=LOG_PATH / "lr.log",
        save_path=RESULT_PATH / "lr.json",
        n_trials=NUM_TRIALS,
    ),
    delayed(tune_model_mult_outcomes)(
        model_builder=lightgbm_model_builder,
        model_abrv="lgbm",
        outcome_dict=OUTCOME_DICT,
        scoring="roc_auc",
        log_file_path=LOG_PATH / "lgbm.log",
        save_path=RESULT_PATH / "lgbm.json",
        n_trials=NUM_TRIALS,
    ),
    delayed(tune_model_mult_outcomes)(
        model_builder=svc_model_builder,
        model_abrv="svc",
        outcome_dict=OUTCOME_DICT,
        scoring="roc_auc",
        log_file_path=LOG_PATH / "svc.log",
        save_path=RESULT_PATH / "svc.json",
        n_trials=NUM_TRIALS,
    ),
]
lr_results_dict, lightgbm_results_dict, svc_results_dict = Parallel(n_jobs=-1)(jobs)

Get prelim results

In [ ]:
with open(RESULT_PATH / "lr.json", "r") as f:
    lr_results_dict = json.load(f)
get_prelim_results(
    results_dict=lr_results_dict,
    model_builder=lr_model_builder,
    model_abrv="lr",
    outcome_dict=OUTCOME_DICT,
    model_save_dir=BASE_PATH / "models" / "trained",
)
with open(RESULT_PATH / "lgbm.json", "r") as f:
    lightgbm_results_dict = json.load(f)
get_prelim_results(
    results_dict=lightgbm_results_dict,
    model_builder=lightgbm_model_builder,
    model_abrv="lgbm",
    outcome_dict=OUTCOME_DICT,
    model_save_dir=BASE_PATH / "models" / "trained",
)
with open(RESULT_PATH / "svc.json", "r") as f:
    svc_results_dict = json.load(f)
###### make probability=TRUE for svc
for outcome, sub_dict in svc_results_dict.items():
    print(outcome)
    sub_dict["best_params"]["probability"] = True
    print(sub_dict["best_params"])
get_prelim_results(
    results_dict=svc_results_dict,
    model_builder=svc_model_builder,
    model_abrv="svc",
    outcome_dict=OUTCOME_DICT,
    model_save_dir=BASE_PATH / "models" / "trained",
)